In [21]:
import numpy as np
import numpy.random as random

from deepiv.models import Treatment, Response
import deepiv.architectures as architectures
import deepiv.densities as densities

from keras.layers import Input, Dense
from keras.models import Model
from keras.layers.merge import Concatenate

import data_generator

In [28]:
class DeepIV():
    """Deep Instrumental Variables
    This class is an implemenatation of an approach using
    deep neural networks to estimate the causal effect of treatment T
    using a set of intrumental variables Z.
    
    See https://arxiv.org/abs/1612.09596
    """
    def __init__(self, num_instruments, num_features, num_treatments,
                 hidden_layer_sizes = [128, 64, 32], 
                 num_gaussians = 10, dropout_rate = 0.2,
                 l2_penalty = 0.0001, activation_function = "relu",
                 optimizer = "adam"):
        
        instruments = Input(shape = (num_instruments,), name = "instruments")
        features = Input(shape = (num_features,), name = "features")
        treatments = Input(shape = (num_treatments,), name = "treatment")
        
        instruments_and_features = Concatenate(axis=1)([instruments, features])
        features_and_treatments = Concatenate(axis=1)([features, treatments])
        
        def gaussian_mix_outputs(x):
            densities.mixture_of_gaussian_output(x, num_gaussians)
        
        estimated_treatment = architectures.feed_forward_net(
            instruments_and_features, 
            gaussian_mix_outputs,
            hidden_layers = hidden_layer_sizes, 
            dropout_rate = dropout_rate,
            l2 = l2_penalty, 
            activations = activation_function)
        
        treatment_model = Treatment(inputs=[instruments, features], outputs=estimated_treatment)
        treatment_model.compile(optimizer, loss="mixture_of_gaussians", n_components = num_gaussians)
        
        estimated_response = architectures.feed_forward_net(
            features_and_treatments, 
            Dense(1),
            hidden_layers = hidden_layer_sizes,
            dropout_rate=dropout_rate,
            l2 = l2_penalty,
            activations = activation_function)
        
        response_model = Response(
            treatment = treatment_model,
            inputs = [features, treatment],
            outputs = estimated_response)
        response_model.compile(optimizer, loss = "mse")
        
        self.treatment_model = treatment_model
        self.response_model = response_model
        
        
    def fit(self, Z, X, T, epochs = 300, batch_size = 100, verbose = True):
        self.treatment_model.fit([Z, X], T, epochs=epochs, batch_size=batch_size)
        self.response_model.fit([Z, X], Y, epochs=epochs, batch_size=batch_size, samples_per_batch=2)
        

    def get_represetation(self, X, T):
        return self.response_model.get_represetation([X, T])
    
    def predict(self, X, T):
        return self.response_model.predict([X, T])

    
    

In [29]:
N = 5000
images = False
def datafunction(n, s, images=images, test=False):
    return data_generator.demand(n=n, seed=s, ypcor=0.5, use_images=images, test=test)

x, z, t, y, g_true = datafunction(N, 1)

num_instruments = z.shape[1]
num_features = x.shape[1]
num_treatments = t.shape[1]


In [30]:
deepiv = DeepIV(num_instruments, num_features, num_treatments)

TypeError: Output tensors to a Treatment must be Keras tensors. Found: None

In [33]:
instruments = Input(shape=(z.shape[1],), name="instruments")
features = Input(shape=(x.shape[1],), name="features")
treatment_input = Concatenate(axis=1)([instruments, features])

hidden = [128, 64, 32]

act = "relu"


est_treat = architectures.feed_forward_net(treatment_input, lambda x: densities.mixture_of_gaussian_output(x, 10),
                                           hidden_layers=[128, 64, 32],
                                           dropout_rate=0.5, l2=0.0001,
                                           activations="relu")

treatment_model = Treatment(inputs=[instruments, features], outputs=est_treat)
treatment_model.compile('adam',
                        loss="mixture_of_gaussians",
                        n_components=10)


Sampler inputs: [<tf.Tensor 'instruments_5:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'features_5:0' shape=(?, 8) dtype=float32>]


In [34]:
instruments

<tf.Tensor 'instruments_5:0' shape=(?, 1) dtype=float32>

In [36]:
hidden_layer_sizes = [128, 64, 32]
num_gaussians = 10
dropout_rate = 0.2
l2_penalty = 0.0001
activation_function = "relu"
optimizer = "adam"

instruments = Input(shape = (num_instruments,), name = "instruments")
features = Input(shape = (num_features,), name = "features")
treatments = Input(shape = (num_treatments,), name = "treatment")

instruments_and_features = Concatenate(axis=1)([instruments, features])
features_and_treatments = Concatenate(axis=1)([features, treatments])

def gaussian_mix_outputs(x):
    densities.mixture_of_gaussian_output(x, num_gaussians)

estimated_treatment = architectures.feed_forward_net(
    instruments_and_features, 
    gaussian_mix_outputs,
    hidden_layers = hidden_layer_sizes, 
    dropout_rate = dropout_rate,
    l2 = l2_penalty, 
    activations = activation_function)

treatment_model = Treatment(inputs=[instruments, features], outputs=estimated_treatment)
treatment_model.compile(optimizer, loss="mixture_of_gaussians", n_components = num_gaussians)


TypeError: Output tensors to a Treatment must be Keras tensors. Found: None

In [38]:
instruments_and_features

<tf.Tensor 'concatenate_13/concat:0' shape=(?, 9) dtype=float32>

In [39]:
treatment_input

<tf.Tensor 'concatenate_9/concat:0' shape=(?, 9) dtype=float32>

In [41]:
instruments = Input(shape=(z.shape[1],), name="instruments")
features = Input(shape=(x.shape[1],), name="features")

In [42]:
instruments

<tf.Tensor 'instruments_8:0' shape=(?, 1) dtype=float32>

In [43]:
features

<tf.Tensor 'features_8:0' shape=(?, 8) dtype=float32>

In [44]:
instruments = Input(shape = (num_instruments,), name = "instruments")
features = Input(shape = (num_features,), name = "features")
treatments = Input(shape = (num_treatments,), name = "treatment")

instruments_and_features = Concatenate(axis=1)([instruments, features])

In [45]:
instruments

<tf.Tensor 'instruments_9:0' shape=(?, 1) dtype=float32>

In [46]:
features

<tf.Tensor 'features_9:0' shape=(?, 8) dtype=float32>

In [47]:
instruments_and_features

<tf.Tensor 'concatenate_16/concat:0' shape=(?, 9) dtype=float32>

In [48]:
treatment_input

<tf.Tensor 'concatenate_9/concat:0' shape=(?, 9) dtype=float32>

In [49]:
architectures.feed_forward_net(treatment_input, lambda x: densities.mixture_of_gaussian_output(x, 10),
                                           hidden_layers=[128, 64, 32],
                                           dropout_rate=0.5, l2=0.0001,
                                           activations="relu")

<tf.Tensor 'concatenate_17/concat:0' shape=(?, 30) dtype=float32>

In [52]:
architectures.feed_forward_net(
    instruments_and_features, 
    lambda x: densities.mixture_of_gaussian_output(x, 10),
    hidden_layers = hidden_layer_sizes, 
    dropout_rate = dropout_rate,
    l2 = l2_penalty, 
    activations = activation_function)

<tf.Tensor 'concatenate_19/concat:0' shape=(?, 30) dtype=float32>

In [51]:
gaussian_mix_outputs

<function __main__.gaussian_mix_outputs>